In [7]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

from bravado.client import SwaggerClient
from bravado.exception import HTTPError
import helper

In [2]:
# Lookup type and region info form the SDE
sde_client = SwaggerClient.from_url("http://evekit-sde.orbital.enterprises/latest/swagger.json",
                                   config={'ues_models': False, 'validate_responses' : False})

In [ ]:
string = "bob has {0} words".format(5)
print(string)

In [ ]:
# Look up Tritanium and The Forge
name = "Tritanium"
sde_client.Inventory.getTypes(typeName=f"{{values: ['%s']}}"%name).result()

# If we only wanted the typeID we coulde have done:
#  ...result()[0]['typeID']

In [ ]:
# Now lookup The Forge
sde_client.Map.getRegions(regionName="{values: ['The Forge']}").result()

In [ ]:
# We only need the typeID and regionID though
type_id = sde_client.Inventory.getTypes(typeName="{values: ['Zydrine']}").result()[0]['typeID']
region_id = sde_client.Map.getRegions(regionName="{values: ['The Forge']}").result()[0]['regionID']

In [ ]:
# Create date range
date_range = pd.date_range(datetime.date.today() - datetime.timedelta(days=2), datetime.date.today())
date_range

In [3]:
# Test the Orbital Enterprises market data service
#  Fetch daily market averages for every date in our date range

# We need a SwaggerClient for the market data colleciton service
mdc_client = SwaggerClient.from_url("https://evekit-market.orbital.enterprises//swagger",
                                   config={'use_models': False, 'validate_responses' : False})

In [ ]:
# Test the client on the first date in our range
mdc_client.MarketData.history(typeID=type_id, regionID=region_id, date=str(date_range[0])).result()

In [ ]:
# Loop through the date range
market_history = []
for next in date_range:
    try:
        print(".", end="")
        next_data = mdc_client.MarketData.history(typeID=type_id, regionID=region_id, date=str(next)).result()
        market_history.append(next_data)
    except HTTPError:
        pass

print()
market_history

In [ ]:
raw_time = market_history[0]['date']
datetime.datetime.utcfromtimestamp(raw_time//1000).replace(microsecond=raw_time%1000*1000)

In [ ]:
def convertRawTime(raw_time):
    return datetime.datetime.utcfromtimestamp(raw_time//1000).replace(microsecond=raw_time%1000*1000)

mh_frame = pd.DataFrame(market_history, index=[convertRawTime(v['date']) for v in market_history])
mh_frame.head()

In [11]:
# Plot it
# mh_frame.avgPrice.plot(title="Tritanium price in The Forge", figsize=[10,5])

In [ ]:
# Look at order books


### Implement my functions

In [4]:
type_id   = helper.get_type_id(sde_client, 'Tritanium')
region_id = helper.get_region_id(sde_client, 'The Forge')

In [8]:
market_history = helper.get_item_history(mdc_client, type_id, region_id, 2)
market_history

...

In [9]:
market_history